In [1]:
mgpath = '/home/gauthier/tools/mg5amcnlo/'
form = 'form'

In [2]:
def convert_model(model_dir):
    
    ## make model compatible with python3 using MG utility
    import sys

    if mgpath not in sys.path:
        sys.path.append(mgpath)
    from madgraph.interface import madgraph_interface as mi

    my_mi = mi.MadGraphCmd()

    my_mi.do_convert_model([model_dir])
    
    ## need more work to fix import statements
    import glob, re

    files = glob.glob(model_dir+'/*.py')
    files.pop(files.index(model_dir+'/write_param_card.py'))
    names = [f.split('/')[-1].split('.')[0] for f in files]

    for file in files:
        text = "from __future__ import absolute_import\n" + open(file).read()
        for lib in names:
            text = re.sub("\n(\s*)import {:}".format(lib), "\n\\1from . import {:}".format(lib), text)
            text = re.sub("\n(\s*)from {:}".format(lib),   "\n\\1from .{:}".format(lib),         text)
        open(file,'w').write(text)

def revert_convert_model(model_dir):

    import glob, re

    files = glob.glob(model_dir+'/*.py')
    names = [f.split('/')[-1].split('.')[0] for f in files]

    for file in files:
        text = open(file).read()
        text = text.replace('from __future__ import absolute_import\n','')
        for lib in names:
            text = re.sub("\n(\s*)from . import {:}".format(lib), "\n\\1import {:}".format(lib), text)
            text = re.sub("\n(\s*)from .{:}".format(lib),   "\n\\1from {:}".format(lib),         text)
        open(file,'w').write(text)

In [3]:
# analyse model
import re, sys, subprocess

def vertices_from_particles(mod,vert):
    vertices = []
    for v in mod.all_vertices:
        if repr(v.particles) == vert:
            vertices.append(v)
    if hasattr(mod,'all_CTvertices'):
        for v in mod.all_CTvertices:
            if repr(v.particles) == vert:
                vertices.append(v)
    return(vertices)

def vertex_to_string(v):
    vertices_strings = []
    for key in v.couplings:
        color   = v.color[key[0]]
        lorentz = v.lorentz[key[1]].structure  if len(key)>1 else '1'
        
        # handle couplings
        if isinstance(v.couplings[key],list):
            coupling = ''
            for c in v.couplings[key]:
                if isinstance(c.value,dict):
                    tmp = '\n\t+'.join(['eps**{:}*({:})'.format(key,c.value[key]) for key in c.value])
                    coupling += '\n\t+({:})'.format(tmp)
                elif isinstance(c.value,str):
                    coupling += '+({:})'.format(c.value)
                else:
                    print('coupling value is neither dictionary or str: {:} = {:}'.format(c,c.value))
                    raise
        elif isinstance(v.couplings[key], mod.object_library.Coupling):
            c = v.couplings[key]
            if isinstance(c.value,dict):
                coupling = '\n\t+'.join(['eps**{:}*({:})'.format(key,c.value[key]) for key in c.value])
            elif isinstance(c.value,str):
                coupling = c.value
            else:
                print('coupling value is neither dictionary or str: {:} = {:}'.format(c,c.value))
        else:
            print('coupling is neither list nor Coupling: {:}'.format(v.couplings[key]))
            raise
            
        vtype = 'TREE' if not hasattr(v,'type') else v.type
        vertex = '\t(' + (')*\n\t('.join([lorentz,color,coupling, vtype])) + ')'
        vertices_strings.append(vertex)
    vertex_string_tot = '\n+'.join(vertices_strings)
    return(vertex_string_tot)

def form_readable_sting(s):
    s = re.sub('([0-9]+)\.000([0-9]+)','decimal(\\1,\\2)/1000',s)
    s = re.sub('([0-9]+)\.00([0-9]+)','decimal(\\1,\\2)/100',s)
    s = re.sub('([0-9]+)\.0([0-9]+)','decimal(\\1,\\2)/10',s)
    s = re.sub('([0-9]+)\.([0-9]+)','decimal(\\1,\\2)',s)
    s = s.replace('.','')
    s = s.replace('_','')
    s = s.replace('**','^')
    return(s)

def functions_and_parameters_in_vertex(vstring):
    functions = set(re.findall('([A-z][^^*() -+;,/?]*)\(',vstring))
    parameters = set(re.findall('[A-z][^^*() -+;,/?]*',vstring))-functions
    return(functions, parameters)

def param_definitions(mod,parameters):
    definitions = {}
    for param in mod.all_parameters:
        if param.name in parameters and param.nature in ['interal', 'internal']:
            definitions[param.name] = param.value
    if hasattr(mod,'all_CTparameters'):
        for param in mod.all_CTparameters:
            if param.name in parameters and param.nature in ['interal', 'internal']:
                definitions[param.name] = '\n\t+'.join(['eps**{:}*({:})'.format(key,param.value[key]) for key in param.value]) \
                    if isinstance(param.value,dict) else param.value
    return(definitions)

def definitions_to_form(definitions):
    dstring = []
    for key in definitions:
        dstring.append('id {:}**a? = ({:})**a;'.format(key,definitions[key]))
    return('\n'.join(dstring))

def get_eftcoeffs(mod):
    eftcoeffs = []
    for p in mod.all_parameters:
        if (p.nature == 'external') and ('DIM6' in p.lhablock or 'aqgc' in p.lhablock) and (p.name != 'Lambda'):
            eftcoeffs.append(p.name)
    return( sorted(eftcoeffs) )

In [4]:
def form_vertex(mod,vert, debug=False, cmd='', pars=[]):    
    # find all vertices with given particles
    vstring = '\n+'.join([vertex_to_string(v) for v in vertices_from_particles(mod,vert)])

    # find functions and parameters
    noexpand = ['MW','G','yb','yt','ee','ee0','v','vev','vev0','sw','cw','sw2','cw2','cw0','sw0'] # won't be expanded
    funs = set()
    params = set()
    defs = []
    defstring = vstring
    while True:
        newfuns,newparams = functions_and_parameters_in_vertex(defstring)
    #     print(newparams)
        defstring = definitions_to_form(param_definitions(mod,[p for p in newparams-params if p not in noexpand]))
        if defstring!='':
            defs.append(defstring)
        funs.update(newfuns)
        if newparams-params == set():
            break
        else:
            params.update(newparams)

    # add and remove some functions and parameters by hand (those used in the form script in particular)
    for p in ['id']:
        if p in params:
            params.remove(p)
    params = list(params) + pars + ['a','b','c', 'zero',
              'Lam', 'Lambda',
              'sw', 'cw',
              'MH', 'MW', 'MZ',
              'ee', 'G', 'aS', 'Gf',
              'cmathpi']
    funs = list(funs) + ['Identity', 'Gamma', 'ProjM', 'ProjP', 'P', 'Metric', 'Epsilon',
              'complex', 'decimal', 'complexconjugate',
              'T', 'pow', 'cmathsqrt']

    # define eft coefficients
    coeffs = [c for c in get_eftcoeffs(mod) if c in params]

    # write form code
    txt = []
    if not debug:
        txt.append('#-')
    txt.append('Format 255;')
    txt.append('Off Stats;')
    txt.append('Symbols UV, R2, TREE;')
    txt.append('Symbols {:};'.format(form_readable_sting(', '.join(sorted(params)))))
    txt.append('CFunctions {:};'.format(form_readable_sting(', '.join(sorted(funs)))))
    
    # factorisation procedure
    txt.append('''
#procedure facth()
    .sort
    CF h;
    S  symb1, symb2;
    collect h;
    factarg h;
    chainout h;
    id h(symb1?) = h(nterms_(symb1),symb1);
    id h(1,symb1?) = symb1;
    id h(symb2?,symb1?) = h(symb1);
#endprocedure
''')
    
    # load expression
    txt.append('Local expr =\n{:};'.format(form_readable_sting(vstring)))
    txt.append('#define coeffs "{:}"'.format(', '.join(coeffs)))

    # replace internal parameters by their definitions
    txt.append('#do dummy = 1,3')
    txt.append(form_readable_sting('\n'.join(defs)))
    txt.append('id complex(0,1) = i_;')
    txt.append('argument;')
    txt.append(form_readable_sting('\n'.join(defs)))
    txt.append('id complex(0,1) = i_;')
    txt.append('endargument;')
    txt.append('#enddo')
    
    # assumes that all variables are real now so that complex conjugation just affects i_
    txt.append('argument complexconjugate;')
    txt.append('id i_ = -i_;')
    txt.append('endargument;')
    txt.append('id complexconjugate(a?) = a;')
    
    # handle decimals
    txt.append('id a?^decimal(0,75) = pow(1/4,a)^3;')
    txt.append('id a?^decimal(1,5)  = pow(1/2,a)^3;')
    txt.append('id a?^(-decimal(2,5))  = pow(-1/2,a)^5;')
    txt.append('id 1/a?^decimal(0,25) = pow(-1/4,a);')
    txt.append('id 1/a?^decimal(0,75) = pow(-1/4,a)^3;')
    txt.append('id 1/a?^decimal(1,5)  = pow(-1/2,a)^3;')
    txt.append('id decimal(0,625) = 5/8;')
    txt.append('id decimal(0,125) = 1/8;')
    txt.append('id decimal(0,25) = 1/4;')
    txt.append('id decimal(0,5) = 1/2;')
    txt.append('id decimal(0,16666666666666666) = 1/6;')
    txt.append('id decimal(0,3333333333333333) = 1/3;')
    
    # handle powers
    txt.append('id cmathsqrt(Gf) = pow(1/2,Gf);')
    txt.append('id 1/cmathsqrt(Gf) = pow(-1/2,Gf);')
    txt.append('id 1/cmathsqrt(2) = pow(-1/2,2);')
    txt.append('repeat id pow(a?,b?)*pow(c?,b?) = pow(a+c,b);')
    txt.append('id pow(0,a?) = 1;')
    txt.append('id pow(b?int_,a?) = a^b;')
    txt.append('id pow(-1/2,2) = pow(1/2,2)/2;')
    
    # simplify redundant projectors
    txt.append('ab ProjM, ProjP;')
    txt.append('#call facth')
    txt.append('#do i=1,8')
    txt.append('#do j=1,4')
    txt.append('id h(ProjM(-\'j\',\'i\')+ProjP(-\'j\',\'i\')) = Identity(-\'j\',\'i\');')
    txt.append('#enddo')
    txt.append('#enddo')
    txt.append('id h(a?) = a;')
    
    # simplify Lorentz structures using gamma^m gamma^n = g(m,n) - i sigma(m,n)
    txt.append('''
.sort
CFunction Sigma;
symbols i1,...,i9, m1,...,m9;
id Sigma(?a) = Sigma(?a)/2; ** UFO convention is non standard sigma = i[g,g] instead of i[g,g]/2????
#do i=1,9
repeat id Gamma?{Gamma,ProjP,ProjM, Sigma, Identity}(?a, 'i',?b) = Gamma(?a,i'i',?b);
repeat id Gamma?{Gamma,ProjP,ProjM, Sigma, Identity}(?a,-'i',?b) = Gamma(?a,m'i',?b);
#enddo
id Gamma?{Gamma,Sigma}(?a,b?)*ProjP?{ProjP,ProjM}(b?,c?) = Gamma(?a,ProjP(c));
id Gamma(m1?,m2?,m3?)*Gamma(m4?,m3?,m5?) = Gamma(m1,m4,m2,m5);
id Gamma(m1?,m1?,m2?,ProjP?(m3?)) = 4*ProjP(m2,m3);
id Gamma(m1?,m2?,i1?,i2?) = Metric(m1,m2)*Identity(i1,i2) - i_*Sigma(m1,m2,i1,i2);
id Metric(m1?,m2?)*Sigma(m1?,m2?,?m3) = 0;
id Metric(m1?,m2?)^2 = 4;
id Identity(i1?,ProjP?(i2?)) = ProjP(i1,i2);
id Gamma(m1?,m2?,m3?)*Identity(m3?{m1,m2,m3},m4?) = Gamma(m1,m2,m4);
''')
    
    # model specific commands
    txt.append(cmd)
    
    # simplity weak mixing angles
    txt.append('''
ab cw,sw;
#call facth
id h( - 1 + cw)*h(1 + cw) = h(-1+cw^2);
id h( - 1 + 2*cw)*h(1 + 2*cw) = h(-1+4*cw^2);
id h(sw - cw)*h(sw + cw) = h(sw^2-cw^2);
argument;
id cw^2 = 1-sw^2;
endargument;
#call facth
id h(-1+sw)*h(1+sw) = -cw^2;
id h(1-sw)*h(1+sw) = cw^2;
id h(a?) = a;
''')
    
    # factorise nicely
    bracket = 'B Identity, Gamma, ProjM, ProjP, P, Metric, T, Sigma;'
    txt.append(bracket)
    txt.append('#call facth')
    txt.append(bracket)
    txt.append('print +s;')
    txt.append('.end')

    txt = '\n'.join(txt)


    # run form
    fname = 'tmp.frm'
    with open(fname,'w') as f:
        f.write(txt)
    run = subprocess.run([form, fname], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    raw = run.stdout.decode('utf8')
    res = re.findall('expr =\s+([^;]+)\s*;',raw)
    if len(res)>0 and not debug:
        res = re.sub('\s*\n\s+','\n',res[0])
        return(res)
    else:
        print(vert)
        print('\t',raw)
        return('')

In [47]:
print(form_vertex(mod,'[c__tilde__, t, a, H]', debug=True, cmd=cmd, pars=pars))

[c__tilde__, t, a, H]
	 FORM 4.3.0 (Nov 12 2022, v4.3.0) 64-bits         Run: Fri Jul 19 15:42:46 2024
    Format 255;
    Off Stats;
    Symbols UV, R2, TREE;
    Symbols CctB, CctG, CctW, CtcB, CtcG, CtcW, G, G, Gf, IC1ctL, IC1ctR, IC1utL, IC
    1utR, IC3ctL, IC3utL, ICctB, ICctB, ICctG, ICctG, ICctW, ICctW, ICctphi, ICeq13,
     ICeq23, ICeu13, ICeu23, IClequS13, IClequS23, IClequS31, IClequS32, IClequT13, 
    IClequT23, IClequT31, IClequT32, IClqM13, IClqM23, IClu13, IClu23, ICtB, ICtG, I
    CtW, ICtcB, ICtcB, ICtcG, ICtcG, ICtcW, ICtcW, ICtcphi, ICtphi, ICuB, ICuG, ICuW
    , ICuphi, Lam, Lambda, Lambda, MH, MT, MUR, MW, MZ, R2, RC1ctL, RC1ctR, RC1utL, 
    RC1utR, RC3ctL, RC3utL, RCctB, RCctB, RCctG, RCctG, RCctW, RCctW, RCctphi, RCeq1
    3, RCeq23, RCeu13, RCeu23, RClequS13, RClequS23, RClequS31, RClequS32, RClequT13
    , RClequT23, RClequT31, RClequT32, RClqM13, RClqM23, RClu13, RClu23, RCtB, RCtG,
     RCtW, RCtcB, RCtcB, RCtcG, RCtcG, RCtcW, RCtcW, RCtcphi, RCtphi, RCuB,

## TopFCNC4f

In [5]:
curr = !pwd
model_dir = curr[0] + "/TopFCNC4f"
convert_model(model_dir)

Note that this is a development version.
This version is intended for development/beta testing and NOT for production.
This version has not been fully tested (if at all) and might have limited user support (if at all)


/home/gauthier/Desktop/top-fcnc/github/TopFCNC4f


In [11]:
revert_convert_model(model_dir)

In [6]:
import TopFCNC4f as mod

In [9]:
vertices = sorted(set(repr(v.particles) for v in mod.all_vertices))

# remove Goldstone vertices for comparison with dim6top
vertices = [v for v in vertices if 'G' not in v]

cmd = '''
id UV = 0; * remove NLO stuff for comparison with dim6top
id R2 = 0;

argument;  * renumber dummy indices, asssuming no clash
id m5 = m1;
id m6 = m2;
endargument;
'''

In [10]:
# dump
expr = {}
for vert in  vertices:
    # flip current orders to match that of dim6top
    if vert == '[e__plus__, e__minus__, t__tilde__, c]':
        expr['[t__tilde__, c, e__plus__, e__minus__]'] = form_vertex(mod,vert,cmd=cmd+'\nmultiply replace_(i1,i3, i3,i1, i2,i4, i4,i2);\n')
        continue
    if vert == '[mu__plus__, mu__minus__, t__tilde__, c]':
        expr['[t__tilde__, c, mu__plus__, mu__minus__]'] = form_vertex(mod,vert,cmd=cmd+'\nmultiply replace_(i1,i3, i3,i1, i2,i4, i4,i2);\n')
        continue
    
    expr[vert] = form_vertex(mod,vert,cmd=cmd)
    
with open('TopFCNC4f_noCT.txt','w') as f:
# with open('TopFCNC4f.txt','w') as f:
    for vert in sorted(expr.keys()):
        f.write(vert)
        f.write('\n')
        f.write(expr[vert])
        f.write('\n')
        f.write('\n')
        

## dim6top

In [32]:
# setup
!wget https://feynrules.irmp.ucl.ac.be/raw-attachment/wiki/dim6top/dim6top_LO_UFO.tar.gz
!tar xzf dim6top_LO_UFO.tar.gz

--2024-07-19 10:48:48--  https://feynrules.irmp.ucl.ac.be/raw-attachment/wiki/dim6top/dim6top_LO_UFO.tar.gz
Resolving feynrules.irmp.ucl.ac.be (feynrules.irmp.ucl.ac.be)... 130.104.48.109
Connecting to feynrules.irmp.ucl.ac.be (feynrules.irmp.ucl.ac.be)|130.104.48.109|:443... connected.
HTTP request sent, awaiting response... 200 Ok
Length: 43552 (43K) [application/x-tar]
Saving to: ‘dim6top_LO_UFO.tar.gz’

dim6top_LO_UFO.tar. 100%[===================>]  42.53K  --.-KB/s    in 0.001s  

2024-07-19 10:48:48 (69.9 MB/s) - ‘dim6top_LO_UFO.tar.gz’ saved [43552/43552]



In [9]:
curr = !pwd
model_dir = curr[0] + "/dim6top_LO_UFO"
convert_model(model_dir)

Note that this is a development version.
This version is intended for development/beta testing and NOT for production.
This version has not been fully tested (if at all) and might have limited user support (if at all)


/home/gauthier/Desktop/top-fcnc/github/dim6top_LO_UFO


In [11]:
# load
import dim6top_LO_UFO as mod

# remove four-fermion vertices that contain either a neutrino or bottom which are not present in TopFCNC4f
vertices = sorted(set(repr(v.particles) for v in mod.all_vertices
                      if not ( v.lorentz[0].name[:4] == 'FFFF' 
                              and ( 'v' in repr(v.particles) or
                                   'b' in repr(v.particles) ) )
                     ))

In [12]:
# a list of four-fermion FCNC operators
sorted(set(repr(v.particles) for v in mod.all_vertices
    if ( v.lorentz[0].name[:4] == 'FFFF' 
        and ( 'v' in repr(v.particles) or 
             'b' in repr(v.particles) ) )
          ))

['[b__tilde__, b, b__tilde__, b]',
 '[b__tilde__, b, b__tilde__, d]',
 '[b__tilde__, b, b__tilde__, s]',
 '[b__tilde__, b, c__tilde__, c]',
 '[b__tilde__, b, u__tilde__, u]',
 '[b__tilde__, b, ve__tilde__, ve]',
 '[b__tilde__, b, vm__tilde__, vm]',
 '[b__tilde__, b, vt__tilde__, vt]',
 '[b__tilde__, c, u__tilde__, d]',
 '[b__tilde__, c, ve__tilde__, e__minus__]',
 '[b__tilde__, c, vm__tilde__, mu__minus__]',
 '[b__tilde__, c, vt__tilde__, ta__minus__]',
 '[b__tilde__, d, s__tilde__, s]',
 '[b__tilde__, d, ve__tilde__, ve]',
 '[b__tilde__, d, vm__tilde__, vm]',
 '[b__tilde__, d, vt__tilde__, vt]',
 '[b__tilde__, s, e__plus__, e__minus__]',
 '[b__tilde__, s, mu__plus__, mu__minus__]',
 '[b__tilde__, s, u__tilde__, u]',
 '[b__tilde__, s, ve__tilde__, ve]',
 '[b__tilde__, s, vm__tilde__, vm]',
 '[b__tilde__, s, vt__tilde__, vt]',
 '[c__tilde__, b, b__tilde__, t]',
 '[c__tilde__, b, d__tilde__, u]',
 '[c__tilde__, b, e__plus__, ve]',
 '[c__tilde__, b, mu__plus__, vm]',
 '[c__tilde__, b, ta_

In [13]:
set(len(v.particles) for v in mod.all_vertices)

{3, 4, 5}

In [14]:
# remove operator coefficients that are not FCNCs
coeffs = [p.name for p in mod.all_parameters if p.lhablock=='DIM6' and p.name!='Lambda']
cmd = []
cmd.append('.sort')
cmd.append('Symbols {:};'.format(', '.join(coeffs)))
cmd.append('#define nofcnc "{:}";'.format(', '.join(coeffs)))
cmd.append('id a?{\'nofcnc\'} = 0;')

# remove yb and ymtau not present in TopFCNC4f
cmd.append('.sort')
cmd.append('Symbols yb, ymtau;')
cmd.append('id yb = 0;')
cmd.append('id ymtau = 0;')

# remove four-quark operators not present in TopFCNC4f
coeffs = [p.name for p in mod.all_parameters if p.lhablock=='FCNC' and ('qq' in p.name or 'qu' in p.name or 'qd' in p.name or 'ud' in p.name or 'uu' in p.name)]
cmd.append('.sort')
cmd.append('Symbols {:};'.format(', '.join(coeffs)))
cmd.append('#define fourquark "{:}";'.format(', '.join(coeffs)))
cmd.append('id a?{\'fourquark\'} = 0;')

# remove bottom operators not present in TopFCNC4f
coeffs = [p.name for p in mod.all_parameters if p.lhablock=='FCNC' and ('bW' in p.name or 'pb' in p.name)]
cmd.append('.sort')
cmd.append('Symbols {:};'.format(', '.join(coeffs)))
cmd.append('#define bottom "{:}";'.format(', '.join(coeffs)))
cmd.append('id a?{\'bottom\'} = 0;')


# remove ptb operators not present in TopFCNC4f
coeffs = [p.name for p in mod.all_parameters if p.lhablock=='FCNC' and ('ptb' in p.name)]
cmd.append('.sort')
cmd.append('Symbols {:};'.format(', '.join(coeffs)))
cmd.append('#define ptb "{:}";'.format(', '.join(coeffs)))
cmd.append('id a?{\'ptb\'} = 0;')


cmd = '\n'.join(cmd)

In [15]:
# dump
expr = {}
for vert in  vertices:
    expr[vert] = form_vertex(mod,vert, cmd=cmd)
    
with open('dim6top.txt','w') as f:
    for vert in expr:
        if expr[vert] == '0':
            continue
        f.write(vert)
        f.write('\n')
        f.write(expr[vert])
        f.write('\n')
        f.write('\n')

# notes

- `dim6top` doesn't use `Sigma` for tensor operators unlike `TopFNC4f`, maybe the convention used in `UFO/MG`  (`σ=i[γ,γ]`?) differs by a factor of 2 compared to the standard one (`σ=i/2[γ,γ]`). This matters for `ctlTIx1x32`, `ctlTx1x32`, `ctlTIx1x23`, `ctlTx1x23`, `ctlTIx1x31`, `ctlTx1x31`, `ctlTIx1x13`, `ctlTx1x13`.

- The `TopFNC4f` implementation of four-fermion operators is only partial. Four-quark operators are absent. Operators with neutrinos or b's too.

- The chirality flipping `ptb` operators are not included either in `TopFNC4f`.

- The bottom and tau Yukawa couplings are set to zero.

- Note the `tsW` operators are generated together with the `tcZ` and `tcγ` ones (idem for the first generation).

- Goldstone bosons are not included in `dim6top`.

## TopFCNC

In [7]:
!wget https://feynrules.irmp.ucl.ac.be/raw-attachment/wiki/TopFCNC/TopFCNC.tar.gz
!tar xzf TopFCNC.tar.gz

--2024-07-19 15:06:44--  https://feynrules.irmp.ucl.ac.be/raw-attachment/wiki/TopFCNC/TopFCNC.tar.gz
Resolving feynrules.irmp.ucl.ac.be (feynrules.irmp.ucl.ac.be)... 130.104.48.109
Connecting to feynrules.irmp.ucl.ac.be (feynrules.irmp.ucl.ac.be)|130.104.48.109|:443... connected.
HTTP request sent, awaiting response... 200 Ok
Length: 254956 (249K) [application/x-tar]
Saving to: ‘TopFCNC.tar.gz’

TopFCNC.tar.gz      100%[===================>] 248.98K  --.-KB/s    in 0.007s  

2024-07-19 15:06:45 (36.1 MB/s) - ‘TopFCNC.tar.gz’ saved [254956/254956]



In [9]:
curr = !pwd
model_dir = curr[0] + "/TopFCNC"
convert_model(model_dir)

Note that this is a development version.
This version is intended for development/beta testing and NOT for production.
This version has not been fully tested (if at all) and might have limited user support (if at all)


/home/gauthier/Desktop/top-fcnc/github/TopFCNC


In [ ]:
#revert_convert_model(model_dir)

In [11]:
import TopFCNC as mod

vertices = sorted(set(repr(v.particles) for v in mod.all_vertices))

# remove Goldstone vertices for comparison with dim6top
vertices = [v for v in vertices if 'G' not in v]

In [26]:
# dictionary to dim6top conventions
dic = [('ctWIx13', 'sw*ctAIx13 + cw*ctZIx13'),
('ctWIx23', 'sw*ctAIx23 + cw*ctZIx23'),
('ctWIx31', 'sw*ctAIx31 + cw*ctZIx31'),
('ctWIx32', 'sw*ctAIx32 + cw*ctZIx32'),
('ctWx13', 'sw*ctAx13 + cw*ctZx13'),
('ctWx23', 'sw*ctAx23 + cw*ctZx23'),
('ctWx31', 'sw*ctAx31 + cw*ctZx31'),
('ctWx32', 'sw*ctAx32 + cw*ctZx32'),
('IC1ctL', '(-1)*cpQMIx32+(-1)*cpQ3Ix32'),
('IC1ctR', '(-1)*cptIx32'),
('IC1utL', '(-1)*cpQMIx31+(-1)*cpQ3Ix31'),
('IC1utR', '(-1)*cptIx31'),
('IC3ctL', '(-1)*cpQ3Ix32'),
('IC3utL', '(-1)*cpQ3Ix31'),
('ICctB', '(cw)/(-ee*sw)*ctZIx32+(-cw**2)/(-ee*sw)*ctWIx32'),
('ICctG', '1/G*ctGIx32'),
('ICctW', '(sw)/(ee)*ctWIx32'),
('ICctphi', 'ctpIx32'),
('ICeq13', '(-1)*cQeIx1x31'),
('ICeq23', '(-1)*cQeIx1x32'),
('ICeu13', '(-1)*cteIx1x31'),
('ICeu23', '(-1)*cteIx1x32'),
('IClequS13', 'ctlSIx1x13'),
('IClequS23', 'ctlSIx1x23'),
('IClequS31', 'ctlSIx1x31'),
('IClequS32', 'ctlSIx1x32'),
('IClequT13', 'ctlTIx1x13'),
('IClequT23', 'ctlTIx1x23'),
('IClequT31', 'ctlTIx1x31'),
('IClequT32', 'ctlTIx1x32'),
('IClqM13', '(-1)*cQlMIx1x31'),
('IClqM23', '(-1)*cQlMIx1x32'),
('IClu13', '(-1)*ctlIx1x31'),
('IClu23', '(-1)*ctlIx1x32'),
('ICtB', '(cw)/(-ee*sw)*ctZIx13+(-cw**2)/(-ee*sw)*ctWIx13'),
('ICtG', '1/G*ctGIx13'),
('ICtW', '(sw)/(ee)*ctWIx13'),
('ICtcB', '(cw)/(-ee*sw)*ctZIx23+(-cw**2)/(-ee*sw)*ctWIx23'),
('ICtcG', '1/G*ctGIx23'),
('ICtcW', '(sw)/(ee)*ctWIx23'),
('ICtcphi', 'ctpIx23'),
('ICtphi', 'ctpIx13'),
('ICuB', '(-cw**2)/(-ee*sw)*ctWIx31+(cw)/(-ee*sw)*ctZIx31'),
('ICuG', '1/G*ctGIx31'),
('ICuW', '(sw)/(ee)*ctWIx31'),
('ICuphi', 'ctpIx31'),
('RC1ctL', 'cpQMx32+cpQ3x32'),
('RC1ctR', 'cptx32'),
('RC1utL', 'cpQ3x31+cpQMx31'),
('RC1utR', 'cptx31'),
('RC3ctL', 'cpQ3x32'),
('RC3utL', 'cpQ3x31'),
('RCctB', '(-cw**2)/(-ee*sw)*ctWx32+(cw)/(-ee*sw)*ctZx32'),
('RCctG', '1/G*ctGx32'),
('RCctW', '(sw)/(ee)*ctWx32'),
('RCctphi', 'ctpx32'),
('RCeq13', 'cQex1x31'),
('RCeq23', 'cQex1x32'),
('RCeu13', 'ctex1x31'),
('RCeu23', 'ctex1x32'),
('RClequS13', 'ctlSx1x13'),
('RClequS23', 'ctlSx1x23'),
('RClequS31', 'ctlSx1x31'),
('RClequS32', 'ctlSx1x32'),
('RClequT13', 'ctlTx1x13'),
('RClequT23', 'ctlTx1x23'),
('RClequT31', 'ctlTx1x31'),
('RClequT32', 'ctlTx1x32'),
('RClqM13', 'cQlMx1x31'),
('RClqM23', 'cQlMx1x32'),
('RClu13', 'ctlx1x31'),
('RClu23', 'ctlx1x32'),
('RCtB', '(cw)/(-ee*sw)*ctZx13+(-cw**2)/(-ee*sw)*ctWx13'),
('RCtG', '1/G*ctGx13'),
('RCtW', '(sw)/(ee)*ctWx13'),
('RCtcB', '(cw)/(-ee*sw)*ctZx23+(-cw**2)/(-ee*sw)*ctWx23'),
('RCtcG', '1/G*ctGx23'),
('RCtcW', '(sw)/(ee)*ctWx23'),
('RCtcphi', 'ctpx23'),
('RCtphi', 'ctpx13'),
('RCuB', '(cw)/(-ee*sw)*ctZx31+(-cw**2)/(-ee*sw)*ctWx31'),
('RCuG', '1/G*ctGx31'),
('RCuW', '(sw)/(ee)*ctWx31'),
('RCuphi', 'ctpx31')]

allids = '\n'.join(['id {:} = {:};'.format(i[0],i[1]) for i in dic])

In [40]:
funs, pars = functions_and_parameters_in_vertex(allids)
pars = sorted(list(pars))
for p in ['id','G','cw','ee','sw']:
    pars.pop(pars.index(p))
pars

['ctpx32',
 'ctpx31',
 'ctpx23',
 'ctpx13',
 'ctpIx32',
 'ctpIx31',
 'ctpIx23',
 'ctpIx13',
 'ctlx1x32',
 'ctlx1x31',
 'ctlTx1x32',
 'ctlTx1x31',
 'ctlTx1x23',
 'ctlTx1x13',
 'ctlTIx1x32',
 'ctlTIx1x31',
 'ctlTIx1x23',
 'ctlTIx1x13',
 'ctlSx1x32',
 'ctlSx1x31',
 'ctlSx1x23',
 'ctlSx1x13',
 'ctlSIx1x32',
 'ctlSIx1x31',
 'ctlSIx1x23',
 'ctlSIx1x13',
 'ctlIx1x32',
 'ctlIx1x31',
 'ctex1x32',
 'ctex1x31',
 'cteIx1x32',
 'cteIx1x31',
 'ctZx32',
 'ctZx31',
 'ctZx23',
 'ctZx13',
 'ctZIx32',
 'ctZIx31',
 'ctZIx23',
 'ctZIx13',
 'ctWx32',
 'ctWx31',
 'ctWx23',
 'ctWx13',
 'ctWIx32',
 'ctWIx31',
 'ctWIx23',
 'ctWIx13',
 'ctGx32',
 'ctGx31',
 'ctGx23',
 'ctGx13',
 'ctGIx32',
 'ctGIx31',
 'ctGIx23',
 'ctGIx13',
 'ctAx32',
 'ctAx31',
 'ctAx23',
 'ctAx13',
 'ctAIx32',
 'ctAIx31',
 'ctAIx23',
 'ctAIx13',
 'cptx32',
 'cptx31',
 'cptIx32',
 'cptIx31',
 'cpQMx32',
 'cpQMx31',
 'cpQMIx32',
 'cpQMIx31',
 'cpQ3x32',
 'cpQ3x31',
 'cpQ3Ix32',
 'cpQ3Ix31',
 'cQlMx1x32',
 'cQlMx1x31',
 'cQlMIx1x32',
 'cQlMIx1x3

In [51]:
cmd = '''
*id UV = 0; * remove NLO stuff for comparison with dim6top
*id R2 = 0;
#do dummy = 1,2
{:}
#enddo
'''.format(allids)

In [52]:
# dump
expr = {}
for vert in  vertices:
    expr[vert] = form_vertex(mod,vert,cmd=cmd,pars=pars)
    
with open('TopFCNC.txt','w') as f:
    for vert in expr:
        f.write(vert)
        f.write('\n')
        f.write(expr[vert])
        f.write('\n')
        f.write('\n')

# notes

- `TopFCNC` just doesn't have the eetc and eetu four-fermion operators